<div style="text-align: right" align="right"><i>Peter Norvig, 3 Jan 2020</i></div>

# Spelling Bee Puzzle

The [3 Jan. 2020 edition of the **538 Riddler**](https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/) concerns the popular NYTimes  [**Spelling Bee**](https://www.nytimes.com/puzzles/spelling-bee) puzzle:

> In this game, seven letters are arranged in a **honeycomb** lattice, with one letter in the center. Here’s the lattice from Dec. 24, 2019:
> 
> <img src="https://fivethirtyeight.com/wp-content/uploads/2020/01/Screen-Shot-2019-12-24-at-5.46.55-PM.png?w=1136" width="150">
> 
> The goal is to identify as many words as possible that meet the following criteria:
> 1. The word must be at least four letters long.
> 2. The word must include the central letter.
> 3. The word cannot include any letter beyond the seven given letters.
>
>Note that letters can be repeated. For example, the words GAME and AMALGAM are both acceptable words. Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, etc. Words that use all seven letters in the honeycomb are known as **pangrams** and earn 7 bonus points (in addition to the points for the length of the word). So in the above example, MEGAPLEX is worth 8 + 7 = 15 points.
>
> ***Which seven-letter honeycomb results in the highest possible score?*** To be a valid choice of seven letters, no letter can be repeated, it must not contain the letter S (that would be too easy) and there must be at least one pangram.
>
> For consistency, please use [this word list](https://norvig.com/ngrams/enable1.txt) to check your game score.



Since the referenced [word list](https://norvig.com/ngrams/enable1.txt) came from [***my*** web site](https://norvig.com/ngrams), I felt somewhat compelled to solve this one.  (Note it is a standard Scrabble® word list that I happen to host a copy of; I didn't curate it.) 

I'll show you how I address the problem. First some imports, then we'll work through 10 steps.

In [1]:
from collections import Counter, defaultdict
from dataclasses import dataclass
from itertools   import combinations
from typing      import List, Set, Dict, Tuple

# 1: Words, Word Scores, and Pangrams

Let's start by defining some basic terms:

- **valid word**: a string of at least 4  letters ('A' to 'Z' but not 'S'), and not more than 7 distinct letters.
- **word list**: a list of valid  words.
- **pangram**: a word with exactly 7 distinct letters.
- **word score**: 1 for a four letter word, or the length of the word for longer words, plus 7 for a pangram.

In [2]:
def is_valid(word) -> bool:
    pass
def word_list(text) -> List[str]: 
    """All the valid words in text (uppercased)."""
    return [w for w in text.upper().split() if is_valid(w)]

def is_pangram(word) -> bool: return len(set(word)) == 7
    pass
def word_score(word) -> int: 
    """The points for this word, including bonus for pangram."""
    return 1 if len(word) == 4 else len(word) + 7 * is_pangram(word)

I'll make a mini word list to experiment with: 

In [3]:
mini = word_list('amalgam amalgamation cacciatore erotica em game gem gems glam megaplex')
mini

['AMALGAM', 'CACCIATORE', 'EROTICA', 'GAME', 'GLAM', 'MEGAPLEX']

Note that `em` and `gem` are too short, `gems` has an `s` which is not allowed, and `amalgamation` has too many distinct letters (8). We're left with six valid words out of the ten candidate words. Here are examples of the other two functions in action:

In [4]:
{w for w in mini if is_pangram(w)}

{'CACCIATORE', 'EROTICA', 'MEGAPLEX'}

In [5]:
{w: word_score(w) for w in mini}

{'AMALGAM': 7,
 'CACCIATORE': 17,
 'EROTICA': 14,
 'GAME': 1,
 'GLAM': 1,
 'MEGAPLEX': 15}

# 2: Honeycombs and Lettersets

A honeycomb lattice consists of (1) a set of seven distinct letters and (2) the one distinguished center letter:


In [6]:
Letter = Letterset = str # Types

@dataclass(frozen=True, order=True)
class Honeycomb:
    """A Honeycomb lattice, with 7 letters, 1 of which is the center."""
    letters: Letterset # 7 letters
    center:  Letter    # 1 letter
        
def letterset(word) -> Letterset:
    """The set of letters in a word, represented as a sorted str."""
    return ''.join(sorted(set(word)))

In [7]:
hc = Honeycomb(letterset('MEGAPLEX'), 'G')
hc

Honeycomb(letters='AEGLMPX', center='G')

The type `Letter` is a `str` of 1 letter and `Letterset` is an unordered collection of letters, which I will represent as a sorted `str`. Why not a Python `set` or `frozenset`? Because a `str` takes up less space in memory, and its printed representation is  easier to read when debugging. Compare:
- `frozenset({'A', 'E', 'G', 'L', 'M', 'P', 'X'})`
- `'AEGLMPX'`

Why sorted? So that  equal lettersets are equal:

In [8]:
assert letterset('EROTICA') == letterset('CACCIATORE') == 'ACEIORT'

# 3: Game Score

The **game score** for a honeycomb is the sum of the word scores for all the words that the honeycomb **can make**. 

A honeycomb can make a word if
(1) the word contains the honeycomb's center, and
(2) every letter in the word is in the honeycomb. 

In [9]:
def game_score(honeycomb, wordlist) -> int:
    pass
def can_make(honeycomb, word) -> bool:
    """Can the honeycomb make this word?"""
    return honeycomb.center in word and all(L in honeycomb.letters for L in word)

In [10]:
{w: word_score(w) for w in mini if can_make(hc, w)}

{'AMALGAM': 7, 'GAME': 1, 'GLAM': 1, 'MEGAPLEX': 15}

In [11]:
assert game_score(hc, mini) == 24 == sum(_.values())

# 4: Top Honeycomb

The strategy for finding the top (highest-scoring) honeycomb is:
 - Compile a list of valid candidate honeycombs.
 - For each honeycomb, compute the game score.
 - Return a (score, honeycomb) tuple with the highest score.

In [12]:
def top_honeycomb(words) -> Tuple[int, Honeycomb]: 
    """Return a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    return max((game_score(h, words), h) 
               for h in candidate_honeycombs(words))

What are the possible candidate honeycombs? We can put any letter (except 'S') in the center, then any 6 remaining letters around the outside;  this gives  25 × (24 choose 6) = 3,364,900 possible honeycombs. It would take hours to apply `game_score` to all of these.

Fortunately, we can use the constraint that a valid honeycomb **must make at least one pangram**.  So the letters of any valid honeycomb must ***be*** the letterset of some pangram (and the center can be any of those letters):

In [13]:
def candidate_honeycombs(words) -> List[Honeycomb]:
    pass
def pangram_lettersets(words) -> Set[Letterset]:
    """All lettersets from the pangram words."""
    return {letterset(w) for w in words if is_pangram(w)}

In [14]:
pangram_lettersets(mini)

{'ACEIORT', 'AEGLMPX'}

In [15]:
candidate_honeycombs(mini) # 2×7 of them

[Honeycomb(letters='AEGLMPX', center='A'),
 Honeycomb(letters='AEGLMPX', center='E'),
 Honeycomb(letters='AEGLMPX', center='G'),
 Honeycomb(letters='AEGLMPX', center='L'),
 Honeycomb(letters='AEGLMPX', center='M'),
 Honeycomb(letters='AEGLMPX', center='P'),
 Honeycomb(letters='AEGLMPX', center='X'),
 Honeycomb(letters='ACEIORT', center='A'),
 Honeycomb(letters='ACEIORT', center='C'),
 Honeycomb(letters='ACEIORT', center='E'),
 Honeycomb(letters='ACEIORT', center='I'),
 Honeycomb(letters='ACEIORT', center='O'),
 Honeycomb(letters='ACEIORT', center='R'),
 Honeycomb(letters='ACEIORT', center='T')]

Now we're ready to find the highest-scoring honeycomb with the mini word list:

In [16]:
top_honeycomb(mini)

(31, Honeycomb(letters='ACEIORT', center='T'))

**It works.** But that's just the mini word list. 

# 5: The enable1 Word List

Here's the real word list, `enable1.txt`, and some counts derived from it:

In [17]:
! [ -e enable1.txt ] || curl -O http://norvig.com/ngrams/enable1.txt
! head enable1.txt

aa
aah
aahed
aahing
aahs
aal
aalii
aaliis
aals
aardvark


In [18]:
! wc -w enable1.txt

  172820 enable1.txt


In [19]:
enable1 = word_list(open('enable1.txt').read())

len(enable1)

44585

In [20]:
len([w for w in enable1 if is_pangram(w)])

14741

In [21]:
len(pangram_lettersets(enable1))

7986

In [22]:
len(candidate_honeycombs(enable1))

55902

To summarize, there are:

- 172,820 words in the `enable1` word list
- 44,585 valid Spelling Bee words
- 14,741 pangram words 
- 7,986 distinct pangram lettersets
- 55,902 (7 × 7,986) candidate pangram-containing honeycombs
- out of 3,364,900 theoretically possible honeycombs

How long will it take to run `top_honeycomb(enable1)`? Most of the computation time is in `game_score` (each call has to look at all 44,585 valid words), so let's estimate the total time by first checking how long it takes to compute the game score of a single honeycomb:

In [23]:
%timeit game_score(hc, enable1)

8.48 ms ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Roughly 9 milliseconds on my computer (this may vary). How many seconds to run `game_score` for all 55,902 valid honeycombs?

In [24]:
55902 * 9/1000

503.118

About 500 seconds, or 8 minutes. I could run `top_honeycomb(enable1)`, take a coffee break, come back, and declare victory. 

But I think that a puzzle like this deserves a more elegant solution. I'd like to get the run time under a minute (as is suggested in [Project Euler](https://projecteuler.net/)), and I have an idea how to do it. 

# 6: Faster Algorithm: Points Table

Here's my plan:

1. Keep the same strategy of trying every pangram letterset, but do some precomputation that will make `game_score` much faster.
1. The precomputation is: compute the `letterset` and `word_score` for each word in the word list, and make a table of `{letterset: total_points}` giving the total number of word score points for all the words that correspond to each letterset. I call this a **points table**.
3. These calculations are independent of the honeycomb, so they need be done only once, not 55,902  times. 
4. Every word that a honeycomb can make is formed from a **letter subset** of the honeycomb's 7 letters. A valid letter subset must include the center letter, and may include any non-empty subset of the other 6 letters, so there are 2<sup>6</sup> – 1 = 63 valid letter subsets. 
4. `game_score2` considers each of the 63 letter subsets of a honeycomb, and sums the point table entry for each one. 
5. Thus, `game_score2` iterates over just 63 letter subsets; a big optimization over `game_score`, which iterated over 44,585 words.


Here's the code:

In [25]:
PointsTable = Dict[Letterset, int] # How many points does a letterset score?

def top_honeycomb2(words) -> Tuple[int, Honeycomb]: 
    pass
def tabulate_points(words) -> PointsTable:
    """Return a Counter of {letterset: points} from words."""
    table = Counter()
    for w in words:
        table[letterset(w)] += word_score(w)
    return table

def letter_subsets(honeycomb) -> List[Letterset]:
    pass
def game_score2(honeycomb, points_table) -> int:
    """The total score for this honeycomb, using a points table."""
    return sum(points_table[s] for s in letter_subsets(honeycomb))

Let's get a feel for how this works. 

First consider `letter_subsets`. A 4-letter honeycomb makes $2^3 - 1= 7$ subsets; 7-letter honeycombs make $2^6 - 1= 63$ subsets:

In [26]:
letter_subsets(Honeycomb('ALMG', 'G')) 

['AG', 'LG', 'MG', 'ALG', 'AMG', 'LMG', 'ALMG']

Now let's eminded ourselves what `mini` is, and compute `tabulate_points(mini)`:

In [27]:
print('mini =', mini)
tabulate_points(mini)

mini = ['AMALGAM', 'CACCIATORE', 'EROTICA', 'GAME', 'GLAM', 'MEGAPLEX']


Counter({'AGLM': 8, 'ACEIORT': 31, 'AEGM': 1, 'AEGLMPX': 15})

The letterset `'AGLM'` gets 8 points, 7 for AMALGAM and 1 for GLAM.  `'ACEIORT'` gets 31 points, 17 for CACCIATORE and 14 for EROTICA. The other lettersets represent one word each. 

Let's make sure we haven't broken the  `top_honeycomb` function:

In [28]:
assert top_honeycomb(mini) == top_honeycomb2(mini)

# 7: The Solution

Finally, the solution to the puzzle on the real word list:

In [29]:
%time top_honeycomb2(enable1)

CPU times: user 1.65 s, sys: 3.78 ms, total: 1.65 s
Wall time: 1.65 s


(3898, Honeycomb(letters='AEGINRT', center='R'))

**Wow! 3898 is a high score!** And the whole computation took **less than 2 seconds**!

We can see that `game_score2` is about 300 times faster than `game_score`:

In [30]:
points_table = tabulate_points(enable1)

%timeit game_score2(hc, points_table)

26.4 µs ± 90.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# 8: Even Faster Algorithm: Branch and Bound

A run time of less than 2 seconds is pretty good! But I'm not ready to stop now.

Consider the word **JUKEBOX**. It is a pangram, but what with the **J**, **K**,  and **X**, it is a low-scoring honeycomb, regardless of what center is used:

In [31]:
honeycombs = [Honeycomb(letterset('JUKEBOX'), C) for C in 'JUKEBOX']

{h: game_score(h, enable1) for h in honeycombs}

{Honeycomb(letters='BEJKOUX', center='J'): 26,
 Honeycomb(letters='BEJKOUX', center='U'): 32,
 Honeycomb(letters='BEJKOUX', center='K'): 26,
 Honeycomb(letters='BEJKOUX', center='E'): 37,
 Honeycomb(letters='BEJKOUX', center='B'): 49,
 Honeycomb(letters='BEJKOUX', center='O'): 39,
 Honeycomb(letters='BEJKOUX', center='X'): 15}

It would be great if we could determine that **JUKEBOX** is not a top honeycomb in one call to `game_score2`, rather than seven calls. My idea:
- Keep track of the top score found so far.
- For each pangram letterset, ask "if we weren't required to use the center letter, what would this letterset score?"
- Check if that score (which is an upper bound of the score using any one center letter) is higher than the top score so far.
- If yes, then try it with all seven centers; if not then discard it without trying any centers.
- This is called a [**branch and bound**](https://en.wikipedia.org/wiki/Branch_and_bound) algorithm: prune a whole **branch** (of 7 honeycombs) if an upper **bound** can't beat the top score.

To compute the score of a honeycomb with no center, it turns out I can just call `game_score2` on `Honeycomb(letters, '')`. This works because of a quirk of Python:  `game_score2` checks if `honeycomb.center in letters`; normally in Python the expression `x in y` means "is `x` a member of the collection `y`", but when `y` is a string it means "is `x` a substring of `y`", and the empty string is a substring of every string. (If I had represented a letterset as a Python `set`, this wouldn't work.)

Thus, I can rewrite `top_honeycomb` as follows:

In [32]:
def top_honeycomb3(words) -> Tuple[int, Honeycomb]: 
    """Return a (score, honeycomb) tuple with a highest-scoring honeycomb."""
    points_table = tabulate_points(words)
    top_score, top = -1, None
    pangrams = (s for s in points_table if len(s) == 7)
    for p in pangrams:
        if game_score2(Honeycomb(p, ''), points_table) > top_score:
            for center in p:
                honeycomb = Honeycomb(p, center)
                score = game_score2(honeycomb, points_table)
                if score > top_score:
                    top_score, top = score, honeycomb
    return top_score, top

In [33]:
%time top_honeycomb3(enable1)

CPU times: user 360 ms, sys: 2 ms, total: 362 ms
Wall time: 361 ms


(3898, Honeycomb(letters='AEGINRT', center='R'))

Awesome! We get the same answer, and the computation is about 5 times faster; about **1/3 second**.

For how many pangram lettersets did we have to check all 7 centers? We can find out by copy-pasting `top_honeycomb3` and annotating it to keep a `COUNT` of the number of pangrams that are checked, and to print a line of output when a honeycomb (either with or without a center letter) outscores the top score.

In [34]:
def top_honeycomb3_annotated(words) -> Tuple[int, Honeycomb]: 
    pass
top_honeycomb3_annotated(enable1)

ADFLORW scores  333 with center ∅ (pangram  1 (1/7986))
        scores  265 with center A 
ABDENOR scores 1856 with center ∅ (pangram  2 (2/7986))
        scores 1148 with center A 
        scores 1476 with center D 
        scores 1578 with center E 
ABEIRTV scores 1585 with center ∅ (pangram  3 (4/7986))
ABDEORT scores 2434 with center ∅ (pangram  4 (28/7986))
        scores 1679 with center A 
        scores 2134 with center E 
ABCDERT scores 2254 with center ∅ (pangram  5 (35/7986))
ACELNRT scores 2529 with center ∅ (pangram  6 (46/7986))
        scores 2158 with center A 
        scores 2225 with center E 
ACDELRT scores 2746 with center ∅ (pangram  7 (47/7986))
        scores 2273 with center A 
        scores 2608 with center E 
ACENORT scores 2799 with center ∅ (pangram  8 (50/7986))
ACEIPRT scores 2653 with center ∅ (pangram  9 (57/7986))
ACDEIRT scores 3407 with center ∅ (pangram 10 (71/7986))
        scores 3023 with center E 
ACEINRT scores 3575 with center ∅ (pangram 11 (7

(3898, Honeycomb(letters='AEGINRT', center='R'))

Only 14 pangram lettersets had to have all 7 centers checked. We were lucky that the 4th pangram out of 7986, ABDEORT, happened to be aa good one, scoring 2134 points (with center E), setting a high score so that most of the remaining pangrams only needed to be checked with an empty center. The total number of calls to `game_score2` is:

In [35]:
len(pangram_lettersets(enable1)) + 14 * 7

8084

8,084 is a big improvement over 55,902.

# 9: Fancy Report

I'd like to see the actual words that each honeycomb can make, and I'm curious about how the words are divided up by letterset. Here's a function to provide such a report. I remembered that there is a `fill` function in Python (it is in the `textwrap` module) but this turned out to be a lot more complicated than I expected. I guess it is difficult to create a practical extraction and reporting tool. I feel you, [Larry Wall](http://www.wall.org/~larry/).

In [36]:
from textwrap import fill

def report(honeycomb=None, words=enable1):
    pass
def Ns(n, noun):
    """A string with `n` followed by the plural or singular of noun:
    Ns(3, 'bear') => '3 bears'; Ns(1, 'world') => '1 world'"""  
    return f"{n:d} {noun}{' ' if n == 1 else 's'}"

def group_by(items, key):
    "Group items into bins of a dict, each bin keyed by key(item)."
    bins = defaultdict(list)
    for item in items:
        bins[key(item)].append(item)
    return bins

In [37]:
report(Honeycomb('AEGLMPX', 'G'), mini)

Honeycomb(letters='AEGLMPX', center='G') scores 24 points on 4 words from a 6 word list:

AEGLMPX  15 points 1 pangram  MEGAPLEX(15)
   AEGM   1 point   1 word  GAME(1)
   AGLM   8 points  2 words AMALGAM(7) GLAM(1)


In [38]:
report(words=enable1)

Top Honeycomb(letters='AEGINRT', center='R') scores 3898 points on 537 words from a 44585 word list:

AEGINRT 832 points 50 pangrams AERATING(15) AGGREGATING(18) ARGENTINE(16) ARGENTITE(16) ENTERTAINING(19)
        ENTRAINING(17) ENTREATING(17) GARNIERITE(17) GARTERING(16) GENERATING(17) GNATTIER(15) GRANITE(14)
        GRATINE(14) GRATINEE(15) GRATINEEING(18) GREATENING(17) INGRATE(14) INGRATIATE(17) INTEGRATE(16)
        INTEGRATING(18) INTENERATING(19) INTERAGE(15) INTERGANG(16) INTERREGNA(17) INTREATING(17)
        ITERATING(16) ITINERATING(18) NATTERING(16) RATTENING(16) REAGGREGATING(20) REATTAINING(18)
        REGENERATING(19) REGRANTING(17) REGRATING(16) REINITIATING(19) REINTEGRATE(18) REINTEGRATING(20)
        REITERATING(18) RETAGGING(16) RETAINING(16) RETARGETING(18) RETEARING(16) RETRAINING(17)
        RETREATING(17) TANGERINE(16) TANGIER(14) TARGETING(16) TATTERING(16) TEARING(14) TREATING(15)
 AEGINR 270 points 35 words AGINNER(7) AGREEING(8) ANEARING(8) ANERGIA(7) ANGER

# 10: 'S' Words

What if we allowed honeycombs and words to have an 'S' in them? I'll make a new word list, and report on it:

In [39]:
enable1s = [w for w in open('enable1.txt').read().upper().split()
            if len(w) >= 4 and len(set(w)) <= 7]

report(words=enable1s)

Top Honeycomb(letters='AEINRST', center='E') scores 8681 points on 1179 words from a 98141 word list:

AEINRST 1381 points 86 pangrams ANESTRI(14) ANTISERA(15) ANTISTRESS(17) ANTSIER(14) ARENITES(15) ARSENITE(15)
        ARSENITES(16) ARTINESS(15) ARTINESSES(17) ATTAINERS(16) ENTERTAINERS(19) ENTERTAINS(17) ENTRAINERS(17)
        ENTRAINS(15) ENTREATIES(17) ERRANTRIES(17) INERTIAS(15) INSTANTER(16) INTENERATES(18) INTERSTATE(17)
        INTERSTATES(18) INTERSTRAIN(18) INTERSTRAINS(19) INTRASTATE(17) INTREATS(15) IRATENESS(16)
        IRATENESSES(18) ITINERANTS(17) ITINERARIES(18) ITINERATES(17) NASTIER(14) NITRATES(15) RAINIEST(15)
        RATANIES(15) RATINES(14) REATTAINS(16) REINITIATES(18) REINSTATE(16) REINSTATES(17) RESINATE(15)
        RESINATES(16) RESISTANT(16) RESISTANTS(17) RESTRAIN(15) RESTRAINER(17) RESTRAINERS(18) RESTRAINS(16)
        RESTRAINT(16) RESTRAINTS(17) RETAINERS(16) RETAINS(14) RETINAS(14) RETIRANTS(16) RETRAINS(15)
        RETSINA(14) RETSINAS(15) SANITARIES(

Allowing 'S' words more than doubles the score!

Here are pictures for the highest-scoring honeycombs, with and without an S:

<img src="http://norvig.com/honeycombs.png" width="350">
<center>
   537 words &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 1,179 words 
    <br>50 pangrams  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 86 pangrams
    <br>3,898 points &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 8,681 points
    <br>
</center>

# Summary

This notebook showed how to find the highest-scoring honeycomb. Four ideas led to four approaches:

1. **Brute Force Enumeration**: Compute the game score for every possible honeycomb; return the highest-scoring.
2. **Pangram Lettersets**: Compute the game score for just the honeycombs that are pangram lettersets (with all possible centers).
3. **Points Table**: Precompute the score for each letterset; then for each candidate honeycomb, sum the scores of the 63 letter subsets.
4. **Branch and Bound**: Try all 7 centers only for lettersets that score better than the top score so far.

These ideas led to substantial improvements (gain) in number of honeycombs processed, `game_score` run time, and total run time:


|Approach|Honeycombs|Gain|`game_score` Time|Gain|Total Run Time|Gain|
|--------|----------|--------|----|---|---|---|
|**1. Brute Force Enumeration**|3,364,900|——|9000 microseconds|——|8.5 hours|——|
|**2. Pangram Lettersets**|55,902|60×|9000 microseconds|——|500 seconds|60×|
|**3. Points Table**|55,902|60×|25 microseconds|360×|1.6 seconds|20,000×|
|**4. Branch and Bound**|8,084 |400×|25 microseconds|360×|0.36 seconds|80,000×|

